In [63]:
import pandas as pd
import altair as alt
import seaborn as sns
import numpy as np

In [53]:
df = pd.read_hdf("data/watch_minutes.hdf")

In [54]:
df

,user_id,video_id,watched_minutes
0,0,0,1
1,0,39,3
2,0,52,27
3,0,53,85
4,0,62,88
...,...,...,...
29306,12623,116,1
29307,12624,119,32
29308,12625,119,6
29309,12626,53,16


In [85]:
users.shape

(12628, 2)

In [88]:
users = df.groupby('user_id', as_index=False).agg({'video_id': len})
users
# sns.kdeplot(users['video_id'])

,user_id,video_id
0,0,38
1,1,2
2,2,5
3,3,3
4,4,3
...,...,...
12623,12623,1
12624,12624,1
12625,12625,1
12626,12626,1


In [58]:
pivot = df.pivot(index="user_id", columns="video_id", values="watched_minutes")

In [97]:
pivot.head()

video_id,0,1,2,3,4,5,6,7,8,9,...,555,556,557,558,559,560,561,562,563,564
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
pd.notna(pivot).sum(axis=1).sort_values(ascending=False)

user_id
10       179
142      101
1442      63
1432      46
1484      44
        ... 
2831       1
7345       1
2832       1
2833       1
12627      1
Length: 12628, dtype: int64

In [73]:
1 - np.sum(np.sum(pd.isna(pivot))) / (pivot.shape[0] * pivot.shape[1])

0.004108162504450008

7134820